# PRACTICA GUIADA: Introducción al proceso de Feature Selection en SciKit Learn

## 1. Introducción

El objetivo de esta práctica es introducir algunos de los métodos para la selección de features implementados en Scikit-Learn. Nos concentraremos en algunos de los más comunes. Tal y como se ha mencionado en la exposición teórica, existen muchos más métodos. Pueden consultar sus características en la [documentación oficial del Scikit-Learn dedicada al tema](http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).


## 2. "Familias" de técnicas y métodos de feature selection

Existen diferentes "familias" de técnicas de feature selection. Veamos.


### 2.1 Features con baja varianza

Un primer caso (casi trivial) sería la remoción de aquellos features o características que no aporten "información" a nuestro dataset. Una forma de lograr la información es a través de la varianza. 

Entonces, lo que haremos será remover todos aquellos features que tengan baja varianza (es decir, baja variabilidad). Esto haremos definiendo un `threshold` por debajo del cual eliminaremos los features.


In [1]:
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from scipy import random
import numpy as np
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
boston = datasets.load_boston()

df1 = pd.DataFrame(np.concatenate((iris.data, iris.target.reshape(-1,1)),1),
                  columns = iris.feature_names + ['species'])

* Como vemos si aplicásemos un `threshold = 0.5` la única variable que debería ser excluida sería `sepal_width`.

In [3]:
df1.apply(np.var,0)

sepal length (cm)    0.681122
sepal width (cm)     0.186751
petal length (cm)    3.092425
petal width (cm)     0.578532
species              0.666667
dtype: float64

In [4]:
fet_sel = VarianceThreshold(threshold=0.5)
fet_sel.fit(df1)

VarianceThreshold(threshold=0.5)

* Usando el método `get_support` podemos consultar un array booleano que marca las variables que no han sido excluidas.

In [5]:
fet_sel.get_support()

array([ True, False,  True,  True,  True], dtype=bool)

In [6]:
df1.columns[fet_sel.get_support()]

Index(['sepal length (cm)', 'petal length (cm)', 'petal width (cm)',
       'species'],
      dtype='object')

* Si quisiéramos generar un nuevo array con los datos, simplemente podemos llamar al método `.transform(X)`

In [7]:
df1_reduced = pd.DataFrame(fet_sel.transform(df1), columns = df1.columns[fet_sel.get_support()])
df1_reduced.head()

,sepal length (cm),petal length (cm),petal width (cm),species
0,5.1,1.4,0.2,0.0
1,4.9,1.4,0.2,0.0
2,4.7,1.3,0.2,0.0
3,4.6,1.5,0.2,0.0
4,5.0,1.4,0.2,0.0


### 2.2 Feature selection univariada

Tal y como lo indica el nombre, esto métodos buscan seleccionar features, generalmente comparándolas con el target, y utilizando en base a diferente pruebas univariadas (F, chi cuadrado, etc.). Existen diferentes métodos que realizan esto.

#### `SelectKBest`

El primero que veremos (en un contexto de regresión) es `SelectKBest`. Sigamos usando el dataset anterior. Este método toma dos argumentos:

* `score_func`: una función que devuelve algún score entre $X$ y $Y$ -`chi_squared`, `f_classification`, etc.-
* `k`: la cantidad de "mejores" features que serán seleccionadas

Para facilitar el desarrollo hemos generado una función que ejecuta los métodos, extrae los atributos e imprime los resultados. 

* La función toma un dataframe como input
* Luego instancia el método `SelectKBest` con los parámetro `score_finc=f_classif` y `k=2`
* Realiza el `.fit` del método
* Guarda los resultados (`scores`, `pvalues`, `get_support` y las columnas) en un dataframe y lo retorna. 


In [8]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

df2 = pd.DataFrame(np.concatenate((boston.data, boston.target.reshape(-1,1)),1),
                  columns = list(boston.feature_names) + ['price'])

def select_kbest_reg(data_frame, target, k=2):
    """
    Seleccionado K-Best features para regresión
    :param data_frame: Un dataframe con datos
    :param target: target en el dataframe
    :param k: cantidad deseada de features
    :devuelve un dataframe llamado feature_scores con los scores para cada feature
    """
    feat_selector = SelectKBest(f_regression, k=k)
    _ = feat_selector.fit(data_frame.drop(target, axis=1), data_frame[target])
    
    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = feat_selector.scores_
    feat_scores["P Value"] = feat_selector.pvalues_
    feat_scores["Support"] = feat_selector.get_support()
    feat_scores["Attribute"] = data_frame.drop(target, axis=1).columns
    
    return feat_scores

kbest_feat = select_kbest_reg(df2, "price", k=4)
kbest_feat = kbest_feat.sort_values(["F Score", "P Value"], ascending=[False, False])
kbest_feat

,F Score,P Value,Support,Attribute
12,601.617871,5.081103e-88,True,LSTAT
5,471.846740,2.487229e-74,True,RM
10,175.105543,1.609509e-34,True,PTRATIO
2,153.954883,4.900260e-31,True,INDUS
9,141.761357,5.637734e-29,False,TAX
4,112.591480,7.065042e-24,False,NOX
0,88.151242,2.083550e-19,False,CRIM
8,85.914278,5.465933e-19,False,RAD
6,83.477459,1.569982e-18,False,AGE
1,75.257642,5.713584e-17,False,ZN


* Puede verse entonces que si quisiéramos seleccionar las mejores 10 variables estas serían `LSTAT`,`RM`, `PTRATIO` e `INDUS`.

* Hasta aquí solamente ordenamos e imprimimos los resultados del `SelectKBest` ¿Cómo podríamos hacer para realizar efectivamente la selección?

In [9]:
select = kbest_feat.loc[kbest_feat['Support'] == True,'Attribute']
df2_reduced = df2[select]
df2_reduced.head()

,LSTAT,RM,PTRATIO,INDUS
0,4.98,6.575,15.3,2.31
1,9.14,6.421,17.8,7.07
2,4.03,7.185,17.8,7.07
3,2.94,6.998,18.7,2.18
4,5.33,7.147,18.7,2.18


* Si quisiéramos trabajar con un problema de clasificación podríamos simplemente cambiar el `score_func` a alguna función de scoring adecuada: `f_classif`, `chi2` o bien usar alguna de las funciones que scorean en base a métricas tales como `SelectFpr`, `SelectFdr`  

#### SelectKPercentile

Este método selecciona features en base a los percentiles (definidos por el usuario) de los scores máximos.


In [23]:
import pandas as pd
from sklearn.feature_selection import SelectPercentile, f_regression

def select_percentile(data_frame, target, percentile=15):

    feat_selector = SelectPercentile(f_regression, percentile=percentile)
    _ = feat_selector.fit(data_frame.drop(target, axis=1), data_frame[target])
    
    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = feat_selector.scores_
    feat_scores["P Value"] = feat_selector.pvalues_
    feat_scores["Support"] = feat_selector.get_support()
    feat_scores["Attribute"] = data_frame.drop(target, axis=1).columns
    
    return feat_scores

In [24]:
per_feat = select_percentile(df2, "price", percentile=20)
per_feat = per_feat.sort_values(["F Score", "P Value"], ascending=[False, False])
per_feat

,F Score,P Value,Support,Attribute
12,601.617871,5.081103e-88,True,LSTAT
5,471.846740,2.487229e-74,True,RM
10,175.105543,1.609509e-34,True,PTRATIO
2,153.954883,4.900260e-31,False,INDUS
9,141.761357,5.637734e-29,False,TAX
4,112.591480,7.065042e-24,False,NOX
0,88.151242,2.083550e-19,False,CRIM
8,85.914278,5.465933e-19,False,RAD
6,83.477459,1.569982e-18,False,AGE
1,75.257642,5.713584e-17,False,ZN


### 2.3 Eliminación Recursiva de Features (RFE)

Este método toma como input un estimador externo para tratar de cuantificar el peso (la importancia) de cada feature. El método va eliminando features sucesivamente para ir quedándose con sets cada vez más pequeño de features.

En primera instancia, el método entrena el estimador sobre el total de features y se cuantifica la importancia de cada feature a través de algún atributo del tipo `coef_` o `feature_importance`. El feature menos importante  se elimina del set y se vuelve a entrenar con los restantes. El proceso se repite de forma recursiva hasta que se llega al número de features definido previamente.

El método `RFE()` toma los siguientes argumentos:

* `estimator`: el estimador sobre el cual se va a realizar la selección (podría ser cualquier método supervisado que tenga un `.fit` que provea como importancia de cada feature un método `coef_` o `feature_importance`)
* `n_features_to_select`: la cantidad de features que se busca seleccionar
* `steps`: la cantidad de features que se elimina en cada iteración (puede pasarse en relativo o en absoluto).

In [12]:
import pandas as pd

from sklearn.feature_selection import RFE
from sklearn.svm import SVR

def ref_feature_select(data_frame,target_name, n_feats=20):
    estimator = SVR(kernel='linear')
    selector = RFE(estimator, n_features_to_select=n_feats, step = 1)
    _ = selector.fit(data_frame.drop(target_name,axis = 1),data_frame[target_name])

    scores = pd.DataFrame()
    scores["Attribute Name"] = data_frame.drop(target_name,axis = 1).columns
    scores["Ranking"] = selector.ranking_
    scores["Support"] = selector.support_

    return scores

In [16]:
ref_feature_select(df2,'price', n_feats=5).sort_values('Ranking')

,Attribute Name,Ranking,Support
3,CHAS,1,True
4,NOX,1,True
5,RM,1,True
10,PTRATIO,1,True
12,LSTAT,1,True
7,DIS,2,False
0,CRIM,3,False
2,INDUS,4,False
1,ZN,5,False
6,AGE,6,False


* Ahora bien... ¿cuántos features tenemos que seleccionar? Ese es un problema que podemos resolver con... CrossValidation. 

### RFECV

Por suerte, hay un método que nos permite seleccionar mediante CrossValidation la cantidad de features a retener.

El método `RFECV()` toma los siguientes argumentos:
   * `estimator`: análogo a `RFE`, un estimador de aprendizaje supervisado con un atributo `coef_` o `feature_selection` 
   * `steps`: la cantidad de features que se elimina en cada iteración (puede pasarse en relativo o en absoluto).
   * `cv`: determina el método de valdiación cruzada. Puede pasarse un iterador.

In [14]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

In [15]:
kf = KFold(n_splits=10, shuffle = True)
estimator = SVR(kernel='linear')
selector = RFECV(estimator, step = 1, scoring = 'neg_mean_squared_error', verbose=2)
selector.fit(df2.drop('price',axis = 1),df2['price'])

Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting 

RFECV(cv=None,
   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
   n_jobs=1, scoring='neg_mean_squared_error', step=1, verbose=2)

* Podemos conocer cuáles son las variables seleccionadas luego del proceso de validación cruzada:

In [17]:
df2.drop('price', axis=1).loc[:,selector.support_].columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'PTRATIO',
       'LSTAT'],
      dtype='object')

* También podemos ver cuál es el score (crosvalidado) de cada una de las variables:

In [18]:
selector.grid_scores_

array([-101.40168772,  -89.09411031,  -67.77634859,  -66.52954173,
       -124.04062379, -164.55705382,  -98.87268279,  -50.25554629,
        -49.51563171,  -48.69187173,  -51.4192136 ,  -52.79154718,
        -53.47508118])

## 3. Feature Selection como parte de un pipeline



La introducción de un proceso de selección de features dentro de un pipeline es directa:

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [20]:
kf = KFold(n_splits=3, shuffle = True)
estim = SVR(kernel='linear')
select = RFE(estim, step = 1,  verbose = 1)

pipe = Pipeline([
  ('feat_sel', select),
  ('reg', estim)])

param_grid = {'feat_sel__n_features_to_select' : np.arange(1,len(df2.columns)),
    'reg__C' : [0.1, 1, 10]}

estim = GridSearchCV(pipe, param_grid, verbose=1, n_jobs = 3)

estim.fit(df2.drop('price',axis = 1), df2['price'])

Fitting 3 folds for each of 39 candidates, totalling 117 fits
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 11 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimat

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 8 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 7 features.
Fitting estimator with 13 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 13 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting es

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.8min


Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 13 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 11 features.
Fitting estimator with 9 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 7 features.
Fitting estimator with 13 features.
Fittin

Fitting estimator with 11 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 12 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.
Fitting estimator with 13 features.


[Parallel(n_jobs=3)]: Done 117 out of 117 | elapsed: 14.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('feat_sel', RFE(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
  n_features_to_select=None, step=1, verbose=1)), ('reg', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'feat_sel__n_features_to_select': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), 'reg__C': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [ ]:
pd.DataFrame(estim.cv_results_)

In [ ]:
estim.best_estimator_